# Dependency Analysis and Visualization

> Analyze cross-notebook imports and generate Mermaid.js dependency diagrams

In [ ]:
#| default_exp dependencies

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
from pathlib import Path
from typing import List, Dict, Set, Tuple, Optional
from dataclasses import dataclass, field
from nbdev.config import get_config
from cjm_nbdev_overview.core import *
from cjm_nbdev_overview.parsers import *
import re
from collections import defaultdict

## Data Models

In [ ]:
#| export
@dataclass
class ModuleDependency:
    "Represents a dependency between modules"
    source: str                                 # Source module name
    target: str                                 # Target module name
    import_type: str                            # Type of import (from/import)
    imported_names: List[str] = field(default_factory=list)  # Specific names imported

In [ ]:
#| export
@dataclass
class DependencyGraph:
    "Dependency graph for a project"
    modules: Dict[str, ModuleInfo] = field(default_factory=dict)  # Module name -> ModuleInfo
    dependencies: List[ModuleDependency] = field(default_factory=list)  # All dependencies
    
    def add_module(
        self,
        module: ModuleInfo  # TODO: Add description
    ): # Add a module to the graph - TODO: Add type hint
        "Add a module to the dependency graph"
        self.modules[module.name] = module
    
    def add_dependency(
        self,
        dep: ModuleDependency  # TODO: Add description
    ): # Add a dependency - TODO: Add type hint
        "Add a dependency to the graph"
        self.dependencies.append(dep)
    
    def get_module_dependencies(self, module_name: str  # Module to query
                               ) -> List[ModuleDependency]:  # Dependencies
        "Get all dependencies for a specific module"
        return [d for d in self.dependencies if d.source == module_name]
    
    def get_module_dependents(self, module_name: str    # Module to query
                             ) -> List[ModuleDependency]:  # Dependents
        "Get all modules that depend on a specific module"
        return [d for d in self.dependencies if d.target == module_name]

## Import Analysis

In [ ]:
#| export
def extract_project_imports(import_str: str,            # Import statement
                           project_name: str            # Project package name
                           ) -> Optional[ModuleDependency]:  # Dependency if internal
    "Extract project-internal imports from an import statement"
    # Check for 'from X import Y' pattern
    from_match = re.match(r'from\s+(\S+)\s+import\s+(.+)', import_str)
    if from_match:
        module = from_match.group(1)
        imports = from_match.group(2)
        
        # Check if it's a project import
        if module.startswith(project_name):
            # Extract the module name after project name
            module_parts = module.split('.')
            if len(module_parts) > 1:
                target_module = module_parts[1]  # Get the module after project name
                
                # Parse imported names
                imported_names = [name.strip() for name in imports.split(',')]
                # Handle 'import *'
                if imported_names == ['*']:
                    imported_names = ['*']
                
                return ModuleDependency(
                    source="",  # Will be filled by caller
                    target=target_module,
                    import_type="from",
                    imported_names=imported_names
                )
    
    # Check for 'import X' pattern
    import_match = re.match(r'import\s+(\S+)', import_str)
    if import_match:
        module = import_match.group(1)
        
        # Check if it's a project import
        if module.startswith(project_name):
            module_parts = module.split('.')
            if len(module_parts) > 1:
                target_module = module_parts[1]
                
                return ModuleDependency(
                    source="",  # Will be filled by caller
                    target=target_module,
                    import_type="import",
                    imported_names=[target_module]
                )
    
    return None

In [ ]:
#| export
def analyze_module_dependencies(module: ModuleInfo,     # Module to analyze
                               project_name: str        # Project package name
                               ) -> List[ModuleDependency]:  # Dependencies found
    "Analyze a module's imports to find project-internal dependencies"
    dependencies = []
    
    for import_str in module.imports:
        dep = extract_project_imports(import_str, project_name)
        if dep:
            dep.source = module.name
            dependencies.append(dep)
    
    return dependencies

## Building Dependency Graph

In [ ]:
#| export
def build_dependency_graph(path: Path = None,           # Project path
                          project_name: Optional[str] = None  # Override project name
                          ) -> DependencyGraph:         # Dependency graph
    "Build a dependency graph for all modules in a project"
    if path is None:
        cfg = get_config()
        path = cfg.nbs_path
    
    # Get project name from config if not provided
    if project_name is None:
        cfg = get_config()
        project_name = cfg.lib_name.replace('-', '_')
    
    graph = DependencyGraph()
    
    # Get all notebooks
    notebooks = get_notebook_files(path, recursive=True)
    
    # Parse each notebook and analyze dependencies
    for nb_path in notebooks:
        # Skip index notebooks
        if nb_path.stem in ['index', '00_index']:
            continue
        
        try:
            # Parse the notebook
            module_info = parse_notebook(nb_path)
            graph.add_module(module_info)
            
            # Analyze dependencies
            dependencies = analyze_module_dependencies(module_info, project_name)
            for dep in dependencies:
                graph.add_dependency(dep)
        
        except Exception as e:
            print(f"Error processing {nb_path}: {e}")
            continue
    
    return graph

## Mermaid.js Diagram Generation

In [ ]:
#| export
def generate_mermaid_diagram(graph: DependencyGraph,    # Dependency graph
                           direction: str = "TD",       # Diagram direction (TD/LR)
                           show_imports: bool = False   # Show imported names
                           ) -> str:                    # Mermaid diagram code
    "Generate a Mermaid.js dependency diagram from a dependency graph"
    
    # Define known Mermaid reserved keywords that need escaping
    RESERVED_KEYWORDS = {
        'style', 'end', 'default', 'class', 'classDef', 'click', 'graph', 
        'subgraph', 'direction', 'TD', 'LR', 'TB', 'RL', 'BT'
    }
    
    def escape_node_name(
        name: str  # TODO: Add description
    ) -> str:  # TODO: Add return description
        "Escape node names that conflict with Mermaid reserved keywords"
        if name.lower() in RESERVED_KEYWORDS:
            # Add suffix to avoid keyword conflicts
            return name + "_mod"
        return name
    
    lines = []
    lines.append(f"```mermaid")
    lines.append(f"graph {direction}")
    
    # Add nodes with descriptions
    for module_name, module_info in graph.modules.items():
        escaped_name = escape_node_name(module_name)
        # Create node label with original module name for display
        if module_info.title:
            label = f"{escaped_name}[{module_name}<br/>{module_info.title}]"
        else:
            label = f"{escaped_name}[{module_name}]"
        lines.append(f"    {label}")
    
    lines.append("")  # Empty line for readability
    
    # Consolidate dependencies between same source and target
    dep_map = defaultdict(list)
    for dep in graph.dependencies:
        key = (dep.source, dep.target)
        dep_map[key].extend(dep.imported_names)
    
    # Add edges
    for (source, target), imported_names in dep_map.items():
        escaped_source = escape_node_name(source)
        escaped_target = escape_node_name(target)
        
        if show_imports and imported_names:
            # Remove duplicates and show what's imported
            unique_imports = list(dict.fromkeys(imported_names))  # Preserve order while removing duplicates
            imports = ', '.join(unique_imports[:3])  # Limit to 3
            if len(unique_imports) > 3:
                imports += '...'
            lines.append(f'    {escaped_source} -->|"{imports}"| {escaped_target}')
        else:
            lines.append(f"    {escaped_source} --> {escaped_target}")
    
    # Add styling
    lines.append("")
    lines.append("    classDef default fill:#f9f9f9,stroke:#333,stroke-width:2px")
    
    lines.append("```")
    
    return '\n'.join(lines)

In [ ]:
#| export
def generate_dependency_matrix(graph: DependencyGraph   # Dependency graph
                              ) -> str:                 # Markdown table
    "Generate a dependency matrix showing which modules depend on which"
    # Get all module names sorted
    modules = sorted(graph.modules.keys())
    
    if not modules:
        return "No modules found"
    
    # Build dependency map
    dep_map = defaultdict(set)
    for dep in graph.dependencies:
        dep_map[dep.source].add(dep.target)
    
    # Create table header
    lines = []
    header = "| Module | " + " | ".join(modules) + " |"
    separator = "|--------|" + "".join(["----|" for _ in modules])
    
    lines.append(header)
    lines.append(separator)
    
    # Create rows
    for source in modules:
        row = f"| {source} |"
        for target in modules:
            if target in dep_map[source]:
                row += " ✓ |"
            else:
                row += "   |"
        lines.append(row)
    
    return '\n'.join(lines)

## Testing

Let's test the dependency analysis on our project:

In [ ]:
# Build dependency graph
graph = build_dependency_graph()
print(f"Found {len(graph.modules)} modules with {len(graph.dependencies)} dependencies")

Found 7 modules with 19 dependencies


In [ ]:
# Show dependencies for each module
for module_name in sorted(graph.modules.keys()):
    deps = graph.get_module_dependencies(module_name)
    if deps:
        print(f"\n{module_name} depends on:")
        for dep in deps:
            print(f"  - {dep.target}: {', '.join(dep.imported_names)}")


api_docs depends on:
  - dependencies: *
  - parsers: *
  - core: *
  - tree: *

cli depends on:
  - parsers: *
  - api_docs: *
  - tree: *
  - dependencies: *

dependencies depends on:
  - parsers: ModuleInfo
  - dependencies: ModuleDependency
  - dependencies: generate_mermaid_diagram
  - parsers: *
  - dependencies: DependencyGraph
  - core: *

generators depends on:
  - core: *
  - tree: *

parsers depends on:
  - tree: extract_notebook_info
  - core: *

tree depends on:
  - core: *


In [ ]:
# Test with a sample graph that includes reserved keywords
from cjm_nbdev_overview.dependencies import DependencyGraph, ModuleDependency, generate_mermaid_diagram
from cjm_nbdev_overview.parsers import ModuleInfo
from pathlib import Path

# Create test graph with reserved keywords
test_graph = DependencyGraph()

# Add modules including ones with reserved keywords
modules_data = [
    ("colors", "Colors"),
    ("core", "Core"),
    ("examples", "Practical Usage Examples"),
    ("layout", "Layout"),
    ("modern", "Modern"),
    ("style", "Style"),  # This should trigger the reserved keyword handling
    ("types", "Type Definitions"),
    ("validation", "Validation"),
    ("variants", "Variants")
]

for name, title in modules_data:
    module = ModuleInfo(
        path=Path(f"nbs/{name}.ipynb"),
        name=name,
        title=title,
        description=None,
        functions=[],
        classes=[],
        variables=[],
        imports=[]
    )
    test_graph.add_module(module)

# Add some test dependencies
test_dependencies = [
    ("colors", "validation"),
    ("colors", "types"),
    ("core", "types"),
    ("core", "validation"),
    ("examples", "validation"),
    ("examples", "variants"),
    ("examples", "core"),
    ("examples", "modern"),
    ("examples", "colors"),
    ("examples", "layout"),
    ("examples", "style"),  # This should work with escaped style
    ("layout", "core"),
    ("layout", "types"),
    ("layout", "validation"),
    ("modern", "core"),
    ("style", "core"),      # This should work with escaped style
    ("style", "types"),     # This should work with escaped style
    ("style", "colors"),    # This should work with escaped style
    ("style", "validation"),# This should work with escaped style
    ("variants", "core")
]

for source, target in test_dependencies:
    dep = ModuleDependency(source=source, target=target, import_type="from", imported_names=["*"])
    test_graph.add_dependency(dep)

# Generate diagram that should now work without parse errors
print("## Test Diagram with Reserved Keywords Handled\n")
print(generate_mermaid_diagram(test_graph, direction="LR"))

## Test Diagram with Reserved Keywords Handled

```mermaid
graph LR
    colors[colors<br/>Colors]
    core[core<br/>Core]
    examples[examples<br/>Practical Usage Examples]
    layout[layout<br/>Layout]
    modern[modern<br/>Modern]
    StyleNode[style<br/>Style]
    types[types<br/>Type Definitions]
    validation[validation<br/>Validation]
    variants[variants<br/>Variants]

    colors --> validation
    colors --> types
    core --> types
    core --> validation
    examples --> validation
    examples --> variants
    examples --> core
    examples --> modern
    examples --> colors
    examples --> layout
    examples --> StyleNode
    layout --> core
    layout --> types
    layout --> validation
    modern --> core
    StyleNode --> core
    StyleNode --> types
    StyleNode --> colors
    StyleNode --> validation
    variants --> core

    classDef default fill:#f9f9f9,stroke:#333,stroke-width:2px
```


In [ ]:
# Generate dependency matrix
print("\n## Dependency Matrix\n")
print(generate_dependency_matrix(graph))


## Dependency Matrix

| Module | api_docs | cli | core | dependencies | generators | parsers | tree |
|--------|----|----|----|----|----|----|----|
| api_docs |   |   | ✓ | ✓ |   | ✓ | ✓ |
| cli | ✓ |   |   | ✓ |   | ✓ | ✓ |
| core |   |   |   |   |   |   |   |
| dependencies |   |   | ✓ | ✓ |   | ✓ |   |
| generators |   |   | ✓ |   |   |   | ✓ |
| parsers |   |   | ✓ |   |   |   | ✓ |
| tree |   |   | ✓ |   |   |   |   |


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()